In [4]:
import collections
import numpy as np
import tensorflow as tf

In [17]:
with open("corpus.txt","r",encoding="utf-8") as f:
    corpus = f.read()
raw_corpus = corpus.replace("\n"," <EOS> ")
tokens = raw_corpus.split()

In [19]:
import keras
vocab_size = 2000
text = keras.layers.TextVectorization(max_tokens=vocab_size)
text.adapt(tokens)

In [26]:
text.get_vocabulary()

['',
 '[UNK]',
 'eos',
 '学习',
 '很',
 '深度',
 '有',
 '趣',
 '程序',
 '模型',
 '明天',
 '我们',
 '好',
 '今天',
 '需要',
 '越',
 '设计',
 '机器',
 '智能',
 '数据',
 '我',
 '心情',
 '强大',
 '天气',
 '多',
 '喜欢',
 '写',
 '会',
 '代码',
 '人工',
 '门',
 '适合',
 '进步',
 '语言',
 '训练',
 '让',
 '艺术',
 '自然',
 '耐心',
 '继续',
 '知识',
 '每天',
 '来',
 '更',
 '是',
 '新',
 '效果',
 '改变',
 '技术',
 '怎么样',
 '快',
 '得',
 '应用',
 '广泛',
 '太',
 '处理',
 '在',
 '和',
 '员',
 '可能',
 '变',
 '发展',
 '去',
 '努力',
 '关系',
 '你',
 '优化',
 '人',
 '了',
 '世界',
 '不错',
 '不',
 '下雨',
 '上班',
 '一起',
 '一']

In [28]:
from tensorflow.python.keras.layers import LSTM
from tensorflow.keras.layers import Embedding, Dense
from tensorflow.python.keras import Sequential

window_size = 3
embedding_dim = 64
lstm_units = 128

model = Sequential([
    Embedding(input_dim=len(text.get_vocabulary()), output_dim=embedding_dim, input_length=window_size),
    LSTM(lstm_units, return_sequences=False),
    Dense(len(text.get_vocabulary()), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.build(input_shape=(window_size,len(text.get_vocabulary())))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper_4 (ModuleWrap (3, 76, 64)               4864      
_________________________________________________________________
lstm_2 (LSTM)                (3, 128)                  98816     
_________________________________________________________________
module_wrapper_5 (ModuleWrap (3, 76)                   9804      
Total params: 113,484
Trainable params: 113,484
Non-trainable params: 0
_________________________________________________________________


In [29]:
sequences = text(tokens).numpy()

X = []
y = []
for i in range(len(sequences) - window_size):
    X.append(sequences[i:i + window_size])
    y.append(sequences[i + window_size])

X = np.array(X)
y = np.array(y)

history = model.fit(
    X, y,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

Epoch 1/50
4/4 [==============================] - 3s 158ms/step - loss: 4.3287 - accuracy: 0.0417 - val_loss: 4.3252 - val_accuracy: 0.2258
Epoch 2/50
4/4 [==============================] - 0s 35ms/step - loss: 4.3164 - accuracy: 0.1917 - val_loss: 4.3201 - val_accuracy: 0.1935
Epoch 3/50
4/4 [==============================] - 0s 35ms/step - loss: 4.3040 - accuracy: 0.2167 - val_loss: 4.3144 - val_accuracy: 0.1935
Epoch 4/50
4/4 [==============================] - 0s 35ms/step - loss: 4.2897 - accuracy: 0.2083 - val_loss: 4.3073 - val_accuracy: 0.1935
Epoch 5/50
4/4 [==============================] - 0s 35ms/step - loss: 4.2707 - accuracy: 0.2083 - val_loss: 4.2985 - val_accuracy: 0.1935
Epoch 6/50
4/4 [==============================] - 0s 35ms/step - loss: 4.2469 - accuracy: 0.2083 - val_loss: 4.2867 - val_accuracy: 0.1935
Epoch 7/50
4/4 [==============================] - 0s 34ms/step - loss: 4.2106 - accuracy: 0.1917 - val_loss: 4.2713 - val_accuracy: 0.1935
Epoch 8/50
4/4 [==========

In [57]:
input_text = "我们 一起"
input_tokens = input_text.split()
if len(input_tokens) < window_size:
    input_tokens = [''] * (window_size - len(input_tokens)) + input_tokens
elif len(input_tokens) > window_size:
    input_tokens = input_tokens[-window_size:]

input_sequence = text(input_tokens).numpy().reshape(1, -1)

predictions = model.predict(input_sequence, verbose=0)
next_word = text.get_vocabulary()[predictions.argmax()]
output = input_text + " " + next_word
output

'我们 一起 学习'